# 1.Import data

In [ ]:
import snowflake.connector
from snowflake.connector import cursor
import usaddress
import re
import config_final

In [ ]:
ctx = snowflake.connector.connect(
    user=config_final.user,
    password=config_final.password,
    account=config_final.account
)
cursor = ctx.cursor()

In [ ]:
import pandas as pd
sql_MLS = """"""
df_MLS = pd.read_sql(sql_MLS, con=ctx)

In [ ]:
sql_DATATREE = """"""
df_DATATREE = pd.read_sql(sql_DATATREE, con=ctx)

In [ ]:
df_MLS.head()

In [ ]:
df_DATATREE.head()

In [ ]:
len(df_MLS['MLS_ADDRESS'].unique())/len(df_DATATREE['DATATREE_ADDRESS'].unique())

Max matching ratio couldn't exceed 4%

# 2. First Node - Completely match

In [ ]:
# Drop NaN
df_MLS.dropna(subset=['MLS_ADDRESS'], inplace=True)
df_DATATREE.dropna(subset=['DATATREE_ADDRESS'], inplace=True)


In [ ]:
# Merge the data
df_merge_node1 = pd.merge(df_DATATREE,df_MLS, how = 'left', right_on = ['MLS_ADDRESS', 'POSTALCODE'], 
                          left_on = ['DATATREE_ADDRESS', 'SITUSZIP5'])


In [ ]:
# Compute ratio
first_node_match = len(df_merge_node1[df_merge_node1['MLS_ADDRESS'].notnull()]['MLS_ADDRESS'].unique())
base = len(df_DATATREE['DATATREE_ADDRESS'].unique())
first_node_match_ratio = first_node_match/base
first_node_match_ratio


- Create Matched addresses list

In [ ]:
first_node_match

In [ ]:
MLS_matched_address = []
DATATREE_matched_address = []

In [ ]:
first_node_match_list = list(df_merge_node1[df_merge_node1['MLS_ADDRESS'].notnull()]['MLS_ADDRESS'].unique())
MLS_matched_address.extend(first_node_match_list)
DATATREE_matched_address.extend(first_node_match_list)

In [ ]:
len(MLS_matched_address)

In [ ]:
len(DATATREE_matched_address)

# 3. Second Node - Match after transformation

- Remove punctuation

In [ ]:
# Remove punctuations
from string import punctuation
import re
# punctuation -> space
remove_punctiuation = str.maketrans(punctuation, ' '*len(punctuation))

df_MLS['MLS_ADDRESS'] = df_MLS['MLS_ADDRESS'].apply(lambda x: x.translate(remove_punctiuation))
df_DATATREE['DATATREE_ADDRESS'] = df_DATATREE['DATATREE_ADDRESS'].apply(lambda x: x.translate(remove_punctiuation))
# several spaces -> one space
df_MLS['MLS_ADDRESS'] = df_MLS['MLS_ADDRESS'].apply(lambda x: re.sub(' +', ' ', x))
df_DATATREE['DATATREE_ADDRESS'] = df_DATATREE['DATATREE_ADDRESS'].apply(lambda x: re.sub(' +', ' ', x))


- Create table with splitted addresses

In [ ]:
def get_splited_addr(df,col:str,lt=False):
    import usaddress
    global address
    try:
        address = usaddress.tag(df[col])
    except:
        return None
    if lt:
        return list(address[0])
    else:
        return address
def get_comb_addr(df,splited_col:str):
    test = df[splited_col].to_list()
    values= []
    for i in range(len(test)):
        try:
            values.append(test[i][0])
        except:
            values.append({})
    splited_df = pd.DataFrame(values)
    df.reset_index(inplace=True)
    addr_final = pd.concat([df,splited_df],axis=1)
    del addr_final['index']
    return addr_final

In [ ]:
df1_MLS = df_MLS[:15000] # only take a sample with the first 15000 rows
df1_DATATREE = df_DATATREE[:15000]

In [ ]:
df1_MLS['MLS_ADDRESS_Split'] = df1_MLS.apply(lambda df1_MLS : get_splited_addr(df1_MLS,'MLS_ADDRESS'), axis = 1)
df1_DATATREE['DATATREE_ADDRESS_Split'] = df1_DATATREE.apply(lambda df1_DATATREE : get_splited_addr(df1_DATATREE,'DATATREE_ADDRESS'), axis = 1)
MLS_ADDRESS_final = get_comb_addr(df1_MLS,'MLS_ADDRESS_Split')
DT_ADDRESS_final = get_comb_addr(df1_DATATREE,'DATATREE_ADDRESS_Split')


In [ ]:
MLS_ADDRESS_final.head()

- Address cleaning

In [ ]:
print(DT_ADDRESS_final.columns)
abbr_list = ['StreetNamePreDirectional','StreetNamePostType',
             'OccupancyType','StreetNamePostDirectional']
for col in abbr_list:
    print(f'{col}:\n'+ f'{DT_ADDRESS_final[col].unique()}') 

In [ ]:
print(MLS_ADDRESS_final.columns)
abbr_list = ['StreetNamePreDirectional','StreetNamePostType',
             'OccupancyType','StreetNamePostDirectional']
for col in abbr_list:
    print(f'{col}:\n'+ f'{MLS_ADDRESS_final[col].unique()}') 

In [ ]:
df = pd.read_csv('suffix_abbreviations_converted.csv')
StreetNamePostType_dict = dict(zip(df.common_name, df.abbr))

In [ ]:
StreetNamePostType_dict

In [ ]:
def get_str_replace(MLS_ADDRESS_final,DT_ADDRESS_final):
    #need to change file path
    df = pd.read_csv('suffix_abbreviations_converted.csv')
    StreetNamePostType_dict = dict(zip(df.common_name, df.abbr))
    StreetNameDirectional_dict = {'west': 'w', 'south': 's', 'north': 'n','east': 'e'}

    DT_ADDRESS_final.replace({"StreetNamePreDirectional": StreetNameDirectional_dict},inplace=True)
    DT_ADDRESS_final.replace({"StreetNamePostType": StreetNamePostType_dict},inplace=True)

    MLS_ADDRESS_final.replace({"StreetNamePreDirectional": StreetNameDirectional_dict},inplace=True)
    MLS_ADDRESS_final.replace({"StreetNamePostType": StreetNamePostType_dict},inplace=True)
    return MLS_ADDRESS_final,DT_ADDRESS_final

In [ ]:
# convert address format (direction, abbreviation)
MLS_ADDRESS_final,DT_ADDRESS_final=get_str_replace(MLS_ADDRESS_final,DT_ADDRESS_final)

In [ ]:
print(MLS_ADDRESS_final['StreetNamePreDirectional'].unique())
print(DT_ADDRESS_final['StreetNamePreDirectional'].unique())
print(MLS_ADDRESS_final['StreetNamePostType'].unique())
print(DT_ADDRESS_final['StreetNamePostType'].unique())

In [ ]:
DT_ADDRESS_final[DT_ADDRESS_final['StreetNamePreDirectional'] == '62']

- Merge

In [ ]:
df_merge_node2 = pd.merge(DT_ADDRESS_final, MLS_ADDRESS_final,
                    how = 'left',
                    left_on = ['AddressNumber', 'StreetName','StreetNamePreDirectional','StreetNamePostType',
               'OccupancyIdentifier','OccupancyType', 'SITUSZIP5'],
                    right_on = ['AddressNumber', 'StreetName','StreetNamePreDirectional','StreetNamePostType',
               'OccupancyIdentifier','OccupancyType', 'POSTALCODE'])

In [ ]:
MLS_matched_address.extend(df_merge_node2[df_merge_node2['MLS_ADDRESS'].notnull()]['MLS_ADDRESS'].unique())
MLS_matched_address = list(set(MLS_matched_address))
len(MLS_matched_address)

In [ ]:
DATATREE_matched_address.extend(df_merge_node2[df_merge_node2['MLS_ADDRESS'].notnull()]['DATATREE_ADDRESS'].unique())
DATATREE_matched_address = list(set(DATATREE_matched_address))
len(DATATREE_matched_address)

In [ ]:
second_node_match = len(MLS_matched_address)
second_node_match_ratio = second_node_match/base
second_node_match_ratio

In [ ]:
MLS_unmatched_node2 = MLS_ADDRESS_final[MLS_ADDRESS_final['MLS_ADDRESS'].apply(lambda x: x not in MLS_matched_address)]
DATATREE_unmatched_node2 = DT_ADDRESS_final[DT_ADDRESS_final['DATATREE_ADDRESS'].apply(lambda x: x not in DATATREE_matched_address)]
MLS_unmatched_node2.reset_index(inplace=True)
DATATREE_unmatched_node2.reset_index(inplace=True)
del MLS_unmatched_node2['index']
del DATATREE_unmatched_node2['index']

# 4. Third Node - Match with names

### 4.1 Name cleaning

#### 4.1.1  remove punctuations

In [ ]:
from string import punctuation
import re
# punctuation -> space
def remove_punct(x):
    remove_punctiuation = str.maketrans(punctuation, ' '*len(punctuation))
    if x is not None:
        x = str(x)
        x = x.translate(remove_punctiuation)
        return x
    else:
        return x
MLS_unmatched_node2['NAME'] = MLS_unmatched_node2['NAME'].apply(lambda x: remove_punct(x))
DATATREE_unmatched_node2['OWNERNAME1FULL'] = DATATREE_unmatched_node2['OWNERNAME1FULL'].apply(lambda x: remove_punct(x))
# several spaces -> one space
def conver_space(x):
    if x is not None:
        x = str(x)
        x = re.sub(' +', ' ', x)
        return x.strip()
    else:
        return x
MLS_unmatched_node2['NAME'] = MLS_unmatched_node2['NAME'].apply(lambda x: conver_space(x))
DATATREE_unmatched_node2['OWNERNAME1FULL'] = DATATREE_unmatched_node2['OWNERNAME1FULL'].apply(lambda x: conver_space(x))


#### 4.1.2 Deal with corporation related names

- Compute the frequency of the words

In [ ]:
import nltk
import matplotlib.pyplot as plt
import numpy as np
from nltk.corpus import stopwords

def plot_top_stopwords_barchart(text):
    stop=set(stopwords.words('english'))
    
    new = text.str.split()
    new = new.values.tolist()
    new = list(filter(None,new))

    corpus=[word for i in new for word in i]
    corpus=list(filter(lambda x: len(x) > 1,corpus))#longer than 1
    
    
    from collections import defaultdict
    dic=defaultdict(int)
    for word in corpus:
        if word in stop:
            dic[word]+=1
            
    top=sorted(dic.items(), key=lambda x:x[1],reverse=True)[:10] 
    x,y=zip(*top)
    plt.bar(x,y)

In [ ]:
import seaborn as sns
from nltk.corpus import stopwords
from collections import  Counter

def plot_top_non_stopwords_barchart(text):
    stop=set(stopwords.words('english'))
    
    new= text.str.split()
    new=new.values.tolist()
    new = list(filter(None,new))

    corpus=[word for i in new for word in i]
    corpus=list(filter(lambda x: len(x) > 1,corpus))#longer than 1

    counter=Counter(corpus)
    most=counter.most_common()
    x, y=[], []
    for word,count in most[:50]:
        if (word not in stop):
            x.append(word)
            y.append(count)
    plt.rcParams['figure.figsize']=10,10
    sns.barplot(x=y,y=x)

In [ ]:
plot_top_non_stopwords_barchart(DT_ADDRESS_final['OWNERNAME1FULL'])

In [ ]:
plot_top_non_stopwords_barchart(MLS_ADDRESS_final['NAME'])

In [ ]:
possible_word_list_DT = ['corp', 'llc', 'realty', 'trust', 'inc', 'associates', 'co', 'holdings', 'owner',
                      'properties', 'trustee', 'housing', 'family', 'owners', 'development']
possible_word_list_MLS = ['corp', 'owners', 'llc', 'inc', 'owner', 'realty', 'co', 'tenants', 'agent', 'condo',
                         'house', 'homes', 'housing', 'record', 'estate', 'servicing', 'cor', 'condonminium',
                         'owned', 'portfolio', 'owne', 'trust']
other = ['ltd']

- Replace the words related to corporations

In [ ]:
# Use the words to find possible replacement in sheets
# Incorpate - inc is kind of complicated, so no replacement
replace_name_dict_complicated = {'limited liability company': 'llc', 
                                 'company limited partnership': 'company ltd partnership', 
                                 'co limited partnership': 'co ltd partnership', 'company ltd': 'co ltd',
                                 'associates': 'assoc', 'limited liability partnerships': 'llp',
                                 'properties': 'prop', 'association': 'assn', 'estate': 'est',
                                 'condonminium': 'condo'}

MLS_unmatched_node2.replace({"NAME": replace_name_dict_complicated},inplace=True)
DATATREE_unmatched_node2.replace({"OWNERNAME1FULL": replace_name_dict_complicated},inplace=True)

In [ ]:
replace_name_dict_tail = {'incorporated': 'inc', 'corporation': 'corp', 'authority': 'a'}
def replace_corp(x, replace_dict):
    if x is None:
        return x
    import re
    for key, value in replace_dict.items():
        key = '(.*) ' + key + '$'
        if re.match(key, x):
            return re.search(key, x).group(1) + ' ' + value
    return x

In [ ]:
MLS_unmatched_node2['NAME'] = MLS_unmatched_node2['NAME'].apply(lambda x: replace_corp(x, replace_name_dict_tail))
DATATREE_unmatched_node2['OWNERNAME1FULL'] = DATATREE_unmatched_node2['OWNERNAME1FULL'].apply(lambda  x: replace_corp(str(x), replace_name_dict_tail))

#### 4.1.3 Order the names

In [ ]:
def create_pseudo_name(df_col):
    """
    :param df_col: the name col
    :return:
    """
    assert isinstance(df_col, pd.Series), "It is not a pd.Series"
    df_col = df_col.apply(lambda x: " ".join(sorted(x.split())) if isinstance(x, str) and len(x.split()) <= 3 else x)

    return df_col

In [ ]:
MLS_unmatched_node2['NAME'] = create_pseudo_name(MLS_unmatched_node2['NAME'])
DATATREE_unmatched_node2['OWNERNAME1FULL'] = create_pseudo_name(DATATREE_unmatched_node2['OWNERNAME1FULL'])

### 4.2 Match table to using addresses whose owners only own one address in the DATATREE table

- Find the addresses whose owners only own one address in the DATATREE table

In [ ]:
only_once = DATATREE_unmatched_node2[['DATATREE_ADDRESS','OWNERNAME1FULL']].groupby('OWNERNAME1FULL').count()
only_once = only_once[only_once['DATATREE_ADDRESS']==1]
remaining = only_once[only_once['DATATREE_ADDRESS']> 1]
once_list = list(only_once['DATATREE_ADDRESS'].index)
DATATREE_only_once = DATATREE_unmatched_node2[DATATREE_unmatched_node2['OWNERNAME1FULL'].isin(once_list)]

In [ ]:
df_merge_node3 = pd.merge(DATATREE_only_once, MLS_unmatched_node2, how = 'left', 
                            left_on = ['AddressNumber', 'StreetName', 'StreetNamePreDirectional', 'StreetNamePostType', 'SITUSZIP5'], 
                            right_on = ['AddressNumber', 'StreetName','StreetNamePreDirectional','StreetNamePostType', 'POSTALCODE'])

In [ ]:
df_merge_node3 = df_merge_node3[df_merge_node3['MLS_ADDRESS'].notnull()]

- only contains that one address has apt and the other does not have

In [ ]:
def is_null(x, y):
    if (x == 'nan') and (y == 'nan'):
        return True
    else:
        return False
df_merge_node3['Occupancy_x_none'] = df_merge_node3.apply(lambda x: is_null(str(x['OccupancyType_x']), str(x['OccupancyIdentifier_x'])), axis=1)
df_merge_node3['Occupancy_y_none'] = df_merge_node3.apply(lambda x: is_null(str(x['OccupancyType_y']), str(x['OccupancyIdentifier_y'])), axis=1)

In [ ]:
df_merge_node3 = df_merge_node3[df_merge_node3['Occupancy_y_none'] != df_merge_node3['Occupancy_x_none']]

- Compute the fuzz ratios

In [ ]:
# !pip install thefuzz

In [ ]:
from thefuzz import fuzz
df_merge_node3['NAME_FUZZY_1'] = df_merge_node3.apply(lambda x: fuzz.ratio(str(x['NAME']), str(x['OWNERNAME1FULL'])), axis=1)
df_merge_node3['NAME_FUZZY_sort_1'] = df_merge_node3.apply(lambda x: fuzz.token_sort_ratio(str(x['NAME']), str(x['OWNERNAME1FULL'])), axis=1)

In [ ]:
df_merge_node3[df_merge_node3["NAME"] == df_merge_node3["OWNERNAME1FULL"]][['MLS_ADDRESS', 'DATATREE_ADDRESS','NAME', 'OWNERNAME1FULL']]

- Add address with same names to matched list

In [ ]:
df_merge_node3_matched = df_merge_node3[df_merge_node3["NAME"] == df_merge_node3["OWNERNAME1FULL"]]

In [ ]:
MLS_matched_address.extend(list(df_merge_node3_matched['MLS_ADDRESS'].unique()))
MLS_matched_address = list(set(MLS_matched_address))
len(MLS_matched_address)

In [ ]:
DATATREE_matched_address.extend(list(df_merge_node3_matched['DATATREE_ADDRESS'].unique()))
DATATREE_matched_address = list(set(DATATREE_matched_address))
len(DATATREE_matched_address)

In [ ]:
third_node_match = len(MLS_matched_address)
base = len(df_DATATREE['DATATREE_ADDRESS'].unique())
third_node_match_ratio = third_node_match/base
third_node_match_ratio

- Study fuzzy ratios: there are misspelled names but it's hard to find the threshold

In [ ]:
df_merge_node3 = df_merge_node3[df_merge_node3['MLS_ADDRESS'].notnull()]

In [ ]:
df_merge_node3[(df_merge_node3["NAME_FUZZY_1"] > 75) & (df_merge_node3["NAME_FUZZY_1"] < 100)].sort_values(by="NAME_FUZZY_1" , ascending=False)[['MLS_ADDRESS', 'DATATREE_ADDRESS',
                                                                                                       'NAME', 'OWNERNAME1FULL']]

In [ ]:
df_merge_node3[df_merge_node3["NAME_FUZZY_1"] < 75].sort_values(by="NAME_FUZZY_1" , ascending=False)[['MLS_ADDRESS', 'DATATREE_ADDRESS',
                                                                                                       'NAME', 'OWNERNAME1FULL']].head(20)

- Don't use fuzzy ratios here won't affect much

In [ ]:
len(df_merge_node3[(df_merge_node3["NAME_FUZZY_1"] > 75) & (df_merge_node3["NAME_FUZZY_1"] < 100)]['MLS_ADDRESS'].unique())

# 4. Fourth Node - Match with rooms combined with name fuzzy ratios

In [ ]:
import pandas as pd
sql_MLS = """"""
df_MLS_room = pd.read_sql(sql_MLS, con=ctx)

In [ ]:
sql_DATATREE = """"""
df_DATATREE_room = pd.read_sql(sql_DATATREE, con=ctx)

In [ ]:
df_MLS_room.dropna(subset=['MLS_ADDRESS'], inplace=True)
df_DATATREE_room.dropna(subset=['DATATREE_ADDRESS'], inplace=True)

In [ ]:
MLS_unmatched_node3 = df_MLS_room[df_MLS_room['MLS_ADDRESS'].apply(lambda x: x not in MLS_matched_address)]
DATATREE_unmatched_node3 = df_DATATREE_room[df_DATATREE_room['DATATREE_ADDRESS'].apply(lambda x: x not in DATATREE_matched_address)]

In [ ]:
MLS_unmatched_node3[['ROOMS', 'BEDROOMS', 'FULL_BATHS', 'HALF_BATHS']].value_counts()

- In datatree, almost all of the records are with 0 bedrooms, bathfull, bathspartialnbr; only use totalrooms to match

In [ ]:
DATATREE_unmatched_node3[['BEDROOMS', 'TOTALROOMS', 'BATHFULL', 'BATHSPARTIALNBR']].value_counts()

In [ ]:
MLS_unmatched_node3.reset_index(inplace=True)
DATATREE_unmatched_node3.reset_index(inplace=True)
del MLS_unmatched_node3['index']
del DATATREE_unmatched_node3['index']

In [ ]:
df1_MLS['MLS_ADDRESS_Split'] = df1_MLS.apply(lambda df1_MLS : get_splited_addr(df1_MLS,'MLS_ADDRESS'), axis = 1)
df1_DATATREE['DATATREE_ADDRESS_Split'] = df1_DATATREE.apply(lambda df1_DATATREE : get_splited_addr(df1_DATATREE,'DATATREE_ADDRESS'), axis = 1)
MLS_ADDRESS_final = get_comb_addr(df1_MLS,'MLS_ADDRESS_Split')
DT_ADDRESS_final = get_comb_addr(df1_DATATREE,'DATATREE_ADDRESS_Split')

In [ ]:
df_merge_node4 = pd.merge(DT_ADDRESS_final, MLS_ADDRESS_final, how = 'left', 
                            left_on = ['AddressNumber', 'StreetName', 'StreetNamePreDirectional', 'StreetNamePostType', 'SITUSZIP5'], 
                            right_on = ['AddressNumber', 'StreetName','StreetNamePreDirectional','StreetNamePostType', 'POSTALCODE'])

In [ ]:
df_merge_node4

In [ ]:
def is_null(x, y):
    if (x == 'nan') and (y == 'nan'):
        return True
    else:
        return False
df_merge_node4['Occupancy_x_none'] = df_merge_node4.apply(lambda x: is_null(str(x['OccupancyType_x']), str(x['OccupancyIdentifier_x'])), axis=1)
df_merge_node4['Occupancy_y_none'] = df_merge_node4.apply(lambda x: is_null(str(x['OccupancyType_y']), str(x['OccupancyIdentifier_y'])), axis=1)

In [ ]:
df_merge_node4 = df_merge_node4[df_merge_node4['Occupancy_y_none'] != df_merge_node4['Occupancy_x_none']]

In [ ]:
df_merge_node4 = df_merge_node4[df_merge_node4['MLS_ADDRESS'].notnull()]

- Clean the names

In [ ]:
from settings import *
from Util import *
df_merge_node4['NAME'] =df_merge_node4['NAME'].apply(lambda x: remove_punct(x))
df_merge_node4['OWNERNAME1FULL'] = df_merge_node4['OWNERNAME1FULL'].apply(lambda x: remove_punct(x))
df_merge_node4['NAME'] =df_merge_node4['NAME'].apply(lambda x: conver_space(x))
df_merge_node4['OWNERNAME1FULL'] = df_merge_node4['OWNERNAME1FULL'].apply(lambda x: conver_space(x))
df_merge_node4.replace({"NAME": replace_name_dict_complicated},inplace=True)
df_merge_node4.replace({"OWNERNAME1FULL": replace_name_dict_complicated},inplace=True)
df_merge_node4['NAME'] = df_merge_node4['NAME'].apply(lambda x: replace_corp(x, replace_name_dict_tail))
df_merge_node4['OWNERNAME1FULL'] = df_merge_node4['OWNERNAME1FULL'].apply(lambda  x: replace_corp(str(x), replace_name_dict_tail))

- Compute the fuzz ratios

In [ ]:
df_merge_node4['NAME_FUZZY_1'] = df_merge_node4.apply(lambda x: fuzz.ratio(str(x['NAME']), str(x['OWNERNAME1FULL'])), axis=1)
df_merge_node4['NAME_FUZZY_sort_1'] = df_merge_node4.apply(lambda x: fuzz.token_sort_ratio(str(x['NAME']), str(x['OWNERNAME1FULL'])), axis=1)

In [ ]:
df_merge_node4.columns

In [ ]:
# df_merge_node4[df_merge_node4['TOTALROOMS'] == df_merge_node4['ROOMS']].sort_values(by = 'NAME_FUZZY_1',
#                                                                                    ascending = False)[['MLS_ADDRESS', 'DATATREE_ADDRESS',
#                                                                                                        'NAME', 'OWNERNAME1FULL']].head(20)

- Difficult to confirm whether the address is the same

# 5. Fifth Node - Match with sales information

In [ ]:
import pandas as pd
sql_MLS = """"""
df_MLS_sale = pd.read_sql(sql_MLS, con=ctx)


In [ ]:
sql_DATATREE = """"""
df_DATATREE_sale = pd.read_sql(sql_DATATREE, con=ctx)


In [ ]:
df_MLS_sale.dropna(subset=['MLS_ADDRESS'], inplace=True)
df_DATATREE_sale.dropna(subset=['DATATREE_ADDRESS'], inplace=True)

In [ ]:
print(df_MLS_sale.STATUS.unique())
print(df_MLS_sale.SALE_TYPE.unique())
df_MLS_sale1 = df_MLS_sale.copy()
# filter out "sale" type and "non-lease" type
df_MLS_sale1_filtered = df_MLS_sale1[(df_MLS_sale1['STATUS']=='sale')&(df_MLS_sale1['SALE_TYPE']!='lease')]
df_MLS_sale1_filtered

In [ ]:
MLS_unmatched_node5 = df_MLS_sale1_filtered[df_MLS_sale1_filtered['MLS_ADDRESS'].apply(lambda x: x not in MLS_matched_address)]
DATATREE_unmatched_node5 = df_DATATREE_sale[df_DATATREE_sale['DATATREE_ADDRESS'].apply(lambda x: x not in df_DATATREE_sale)]

# only 1515 rows are what we're looking for 
print(df_MLS_sale.shape)
print(df_MLS_sale1_filtered.shape)


In [ ]:
MLS_unmatched_node5.reset_index(inplace=True)
DATATREE_unmatched_node5.reset_index(inplace=True)

In [ ]:
df1_MLS = MLS_unmatched_node5
df1_DATATREE = DATATREE_unmatched_node5

df1_MLS['MLS_ADDRESS_Split'] = df1_MLS.apply(lambda df1_MLS : get_splited_addr(df1_MLS,'MLS_ADDRESS'), axis = 1)
df1_DATATREE['DATATREE_ADDRESS_Split'] = df1_DATATREE.apply(lambda df1_DATATREE : get_splited_addr(df1_DATATREE,'DATATREE_ADDRESS'), axis = 1)
MLS_ADDRESS_final = get_comb_addr(df1_MLS,'MLS_ADDRESS_Split')
DT_ADDRESS_final = get_comb_addr(df1_DATATREE,'DATATREE_ADDRESS_Split')


In [ ]:
DT_ADDRESS_final = DT_ADDRESS_final.astype(str)
MLS_ADDRESS_final = MLS_ADDRESS_final.astype(str)

In [ ]:
# merge with current recorded date and A-date
df_merge_node5_recent = pd.merge(DT_ADDRESS_final, MLS_ADDRESS_final, how = 'left', 
                            left_on = ['AddressNumber', 'StreetName', 'StreetNamePreDirectional', 'StreetNamePostType', 'SITUSZIP5','CURRENTSALERECORDINGDATE'], 
                            right_on = ['AddressNumber', 'StreetName','StreetNamePreDirectional','StreetNamePostType','POSTALCODE','A_DATE'])


In [ ]:
# match with recent date
df_merge_node5_recent['Occupancy_x_none'] = df_merge_node5_recent.apply(lambda x: is_null(str(x['OccupancyType_x']), str(x['OccupancyIdentifier_x'])), axis=1)
df_merge_node5_recent['Occupancy_y_none'] = df_merge_node5_recent.apply(lambda x: is_null(str(x['OccupancyType_y']), str(x['OccupancyIdentifier_y'])), axis=1)


In [ ]:
df_merge_node5_recent = df_merge_node5_recent[df_merge_node5_recent['Occupancy_y_none'] != df_merge_node5_recent['Occupancy_x_none']]


In [ ]:
df_merge_node5_recent = df_merge_node5_recent[df_merge_node5_recent['MLS_ADDRESS'].notnull()]


In [ ]:
df_merge_node5_recent

In [ ]:
# TODO: landusecode--get distribution, figure out extreme cases -- figure out why missing?

MLS:
* MOST_RECENT_SALE: Supplemental data point for analytics delivery. This is the last known sale for the property (not specifically from MLS)
* MOST_RECENT_SALE_DATE: Supplemental data point for analytics delivery. Date of the last known sale for the property  (not specifically from MLS) 
* PRICE:The price associated with the status, such as sale price or listing price.

Assessor:
* CURRENTSALERECORDINGDATE: Date the transaction was recorded by the county/jurisdiction.
* CURRENTSALESPRICE:Sale price of the recorded transaction
* PREVSALECONTRACTDATE:Date the transaction was recorded by the county/jurisdiction.
* PREVSALESPRICE:Sale price of the recorded transaction


In [ ]:
print(df_MLS_sale.shape)
df_MLS_sale.isna().sum()

In [ ]:
print(df_DATATREE_sale.shape)
df_DATATREE_sale.isna().sum()

In [ ]:
df_MLS_sale[~df_MLS_sale['A_DATE'].isna()]

let's have another experiment! try to merge using different columns

In [ ]:
# just repeat previous cells
sql_MLS = """"""
df_MLS_sale = pd.read_sql(sql_MLS, con=ctx)
sql_DATATREE = """"""
df_DATATREE_sale = pd.read_sql(sql_DATATREE, con=ctx)

df_MLS_sale.dropna(subset=['MLS_ADDRESS'], inplace=True)
df_DATATREE_sale.dropna(subset=['DATATREE_ADDRESS'], inplace=True)
df_MLS_sale1_filtered = df_MLS_sale[(df_MLS_sale['STATUS']=='sale')&(df_MLS_sale['SALE_TYPE']!='lease')]
MLS_unmatched_node5 = df_MLS_sale1_filtered[df_MLS_sale1_filtered['MLS_ADDRESS'].apply(lambda x: x not in MLS_matched_address)]
DATATREE_unmatched_node5 = df_DATATREE_sale[df_DATATREE_sale['DATATREE_ADDRESS'].apply(lambda x: x not in df_DATATREE_sale)]


In [ ]:
MLS_unmatched_node5.reset_index(inplace=True)
DATATREE_unmatched_node5.reset_index(inplace=True)

In [ ]:
df1_DATATREE = DATATREE_unmatched_node5
df1_MLS = MLS_unmatched_node5
df1_MLS['MLS_ADDRESS_Split'] = df1_MLS.apply(lambda df1_MLS : get_splited_addr(df1_MLS,'MLS_ADDRESS'), axis = 1)
df1_DATATREE['DATATREE_ADDRESS_Split'] = df1_DATATREE.apply(lambda df1_DATATREE : get_splited_addr(df1_DATATREE,'DATATREE_ADDRESS'), axis = 1)
MLS_ADDRESS_final = get_comb_addr(df1_MLS,'MLS_ADDRESS_Split')
DT_ADDRESS_final = get_comb_addr(df1_DATATREE,'DATATREE_ADDRESS_Split')


In [ ]:
DT_ADDRESS_final = DT_ADDRESS_final.astype(str)
MLS_ADDRESS_final = MLS_ADDRESS_final.astype(str)

In [ ]:
MLS_ADDRESS_final.MOST_RECENT_SALE_DATE

In [ ]:
# merge with current recorded date and sales date
df_merge_node5_recent = pd.merge(DT_ADDRESS_final, MLS_ADDRESS_final, how = 'left', 
                            left_on = ['AddressNumber', 'StreetName', 'StreetNamePreDirectional', 'StreetNamePostType', 'SITUSZIP5','CURRENTSALERECORDINGDATE'], 
                            right_on = ['AddressNumber', 'StreetName','StreetNamePreDirectional','StreetNamePostType','POSTALCODE','MOST_RECENT_SALE_DATE'])


In [ ]:
# match with recent date

df_merge_node5_recent['Occupancy_x_none'] = df_merge_node5_recent.apply(lambda x: is_null(str(x['OccupancyType_x']), str(x['OccupancyIdentifier_x'])), axis=1)
df_merge_node5_recent['Occupancy_y_none'] = df_merge_node5_recent.apply(lambda x: is_null(str(x['OccupancyType_y']), str(x['OccupancyIdentifier_y'])), axis=1)


In [ ]:
df_merge_node5_recent = df_merge_node5_recent[df_merge_node5_recent['Occupancy_y_none'] != df_merge_node5_recent['Occupancy_x_none']]


In [ ]:
df_merge_node5_recent = df_merge_node5_recent[df_merge_node5_recent['Occupancy_y_none'] != df_merge_node5_recent['Occupancy_x_none']]
df_merge_node5_recent = df_merge_node5_recent[df_merge_node5_recent['MLS_ADDRESS'].notnull()]


In [ ]:
df_merge_node5_recent

In [ ]:
# match with previous date
df_merge_node5_pre = pd.merge(DT_ADDRESS_final, MLS_ADDRESS_final, how = 'left', 
                            left_on = ['AddressNumber', 'StreetName', 'StreetNamePreDirectional', 'StreetNamePostType', 'PREVSALERECORDINGDATE'], 
                            right_on = ['AddressNumber', 'StreetName','StreetNamePreDirectional','StreetNamePostType', 'MOST_RECENT_SALE_DATE'])


In [ ]:

df_merge_node5_pre['Occupancy_x_none'] = df_merge_node5_pre.apply(lambda x: is_null(str(x['OccupancyType_x']), str(x['OccupancyIdentifier_x'])), axis=1)
df_merge_node5_pre['Occupancy_y_none'] = df_merge_node5_pre.apply(lambda x: is_null(str(x['OccupancyType_y']), str(x['OccupancyIdentifier_y'])), axis=1)
df_merge_node5_pre = df_merge_node5_pre[df_merge_node5_pre['Occupancy_y_none'] != df_merge_node5_pre['Occupancy_x_none']]
df_merge_node5_pre = df_merge_node5_pre[df_merge_node5_pre['MLS_ADDRESS'].notnull()]



In [ ]:
df_merge_node5_pre.head(3)

In [ ]:
# price matching
df_merge_node5_price = pd.merge(DT_ADDRESS_final, MLS_ADDRESS_final, how = 'left', 
                            left_on = ['AddressNumber', 'StreetName', 'StreetNamePreDirectional', 'StreetNamePostType','SITUSZIP5', 'CURRENTSALESPRICE'], 
                            right_on = ['AddressNumber', 'StreetName','StreetNamePreDirectional','StreetNamePostType', 'POSTALCODE','PRICE'])


In [ ]:
# match with recent date

df_merge_node5_price['Occupancy_x_none'] = df_merge_node5_price.apply(lambda x: is_null(str(x['OccupancyType_x']), str(x['OccupancyIdentifier_x'])), axis=1)
df_merge_node5_price['Occupancy_y_none'] = df_merge_node5_price.apply(lambda x: is_null(str(x['OccupancyType_y']), str(x['OccupancyIdentifier_y'])), axis=1)


In [ ]:
df_merge_node5_price = df_merge_node5_price[df_merge_node5_price['Occupancy_y_none'] != df_merge_node5_price['Occupancy_x_none']]
df_merge_node5_price = df_merge_node5_price[df_merge_node5_price['MLS_ADDRESS'].notnull()]


In [ ]:
df_merge_node5_price.columns